In [12]:
###############################################################################################################################################
#                                                                                                                                             #
#                                                         Logic and procedure                                                                 #
#                                                                                                                                             #
#       1. We read the datasets for training and testing datasets                                                                             #
#       2. Based on the input from the user, we choose the k value and the distance metric.                                                   #
#       3. We use the 'predict' function to find the predicted outout class value based on the chosen k value and distance value              #
#       4. We use the inbuilt function accuracy_score from sklearn library to find the accuracy for the chosen k value and distance metric    #
#                                                                                                                                             #
#################################################3##############################################################################################



###############################################################################################################################################
#                                                                                                                                             #
#                                                           Functions used                                                                    #
#                                                                                                                                             #
#       1. cosine_similarity = to find the cosine similarity between 2 values                                                                 #
#       2. euclidean_distance = to find the euclidean distance between 2 values                                                               #
#       3. predict = to find the predicted output class value given training and testing dataset and the chosen distance metric and           #
#          k value.                                                                                                                           #
#                                                                                                                                             #
###############################################################################################################################################



###############################################################################################################################################
#                                                                                                                                             #
#                                                           Variables used                                                                    #
#                                                                                                                                             #
#       1. data = training dataset                                                                                                            #
#       2. data1 = testing dataset                                                                                                            #
#       3. target = target class for training dataset                                                                                         #
#       4. target1 = target class for testing dataset                                                                                         #
#       5. op_labels = predicted output class value list                                                                                      #
#       6. distance_value = to choose the distance metric                                                                                     #
#       7. k = chosen k value                                                                                                                 #
#       8. output_prediction = same as op_labels, but in the main code                                                                        #
#       9. A_values = data without target class                                                                                               #
#       10. B_values = data1 without target class                                                                                             #
#       11. distance = to store the distance values for each of the values in the tetsing dataset                                             #
#       12. distance_temp = temp value used to append to distance                                                                             #
#       13. A_norm = normalized values of A_values                                                                                            #
#       14. B_norm = normalized values for B_values                                                                                           #
#       15. dist = argument values for the k minimum values in distance                                                                       #
#       16. labels = target values corresponding to the argument values in dist                                                               #
#       17. lab = most frequent target value for each of the values in the testing dataset                                                    #
#                                                                                                                                             #
###############################################################################################################################################





# Importing all the required libraries 
import numpy as np 
import pandas as pd
from scipy.stats import mode
from sklearn.metrics import accuracy_score


# Function for calculating the cosine similarity between two values
def cosine_similarity (A, B):
  return (A.dot(B)) /(len(A) * len(B))


# Function for calculating the euclidean distance between two values
def euclidean_distance (A, B) :
  return np.sqrt(np.sum((A - B) **2))


# Function for predicting the class of the test dataset. 
def predict(A, B, distance_value, k) : 

  # We assign new variables to store the class of training and testing dataset
  target = A[['class']]
  target1 = B[['class']]

  # We now take the values of both the training and testing dataset(columns 1-8)
  A_values = A[['unif_cell_size','unif_cell_shape','marg_adhesion','single_epith_cell_size','bare_nuclei','bland_chrom','norm_nucleoi','mitoses']]
  B_values = B[['unif_cell_size','unif_cell_shape','marg_adhesion','single_epith_cell_size','bare_nuclei','bland_chrom','norm_nucleoi','mitoses']] 

  A_norm = A[['unif_cell_size','unif_cell_shape','marg_adhesion','single_epith_cell_size','bare_nuclei','bland_chrom','norm_nucleoi','mitoses']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
  B_norm = B[['unif_cell_size','unif_cell_shape','marg_adhesion','single_epith_cell_size','bare_nuclei','bland_chrom','norm_nucleoi','mitoses']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

  # Converting the dataframe Target to an array for data manipulation
  target = np.array(target)

  # We now traverse the testing dataset to find the distances associated for each of the set of values in the testing dataset
  for i in range (len(B_values)) :

    # Initialising the distance list which will contain the distances for the ith dataset values for the testing datset
    distance = []

    # We now take each of the values in the training dataset and compute the distance of them to the ith value in the testing dataset
    for j in range (len(A_values)) : 

      # Based on which type of distance calculation was chosen, we now calculate the distance

      # Cosine similarity case
      if (distance_value == 1) : 
        distance_temp = cosine_similarity(B_values.iloc[i], A_values.iloc[j])
        distance.append(distance_temp)

      # Euclidean distance case
      elif (distance_value == 2) :
        distance_temp = euclidean_distance(B_values.iloc[i], A_values.iloc[j])
        distance.append(distance_temp)

      # Normalized Euclidean distance case
      elif (distance_value == 3) :    
        distance_temp = euclidean_distance(B_norm.iloc[i], A_norm.iloc[j])
        distance.append(distance_temp)

      # In case the value for distance_value was an invalid choice
      else :
        print("Invalid choice for distance value. Try again")

    # We now convert the list 'distance' to an array for data manipulation
    distance = np.array(distance)
    
    # We now use the inbuilt function to find the argument for the k minimum values in the distance array
    # Note that, for cosine similarity, we need to take the max 5 argument values
    if (distance_value == 1) : 
      dist = np.argsort(distance)[:-k] 

    else :
      dist = np.argsort(distance)[:k]

    # We take the target values associated to the minimum arguments
    labels = target[dist]

    # We take the most frequent one(mode) and add it to the variable op_labels. 
    lab = mode(labels) 
    lab = lab.mode[0]
    op_labels.append(lab)
  
  return op_labels


# Main part of the code 
# Reading the training and testing datasets
data=pd.read_csv("train.csv")
data1=pd.read_csv("test.csv")

# We take the class values and assign them seperately
target = data[['class']]
target1 = data1[['class']]

# We convert target1 from dataframe to list for data manipulation
target1.values.tolist()

op_labels = []

# Taking input from the user on what k value and what distance metric to use
distance_value = int(input(" We will be assigning the value \n 1 = Cosine similarity \n 2 = Euclidean Distance \n 3 = normalized euclidean distance. \nChoose the value accordingly\n"))
k = int(input(" \nWe are doing the cases for k = 1, 3, 5, 7. Choose one of these values\n"))

# printing the chosen scenarios
print("\nChosen distance_value is", distance_value)
print("Chosen k value is", k)

# Finding the output predictions for the testing dataset
output_prediction = predict(data, data1, distance_value, k)

# Finding the accuracy for the output predictions we just found
print(accuracy_score(target1, output_prediction))


 We will be assigning the value 
 1 = Cosine similarity 
 2 = Euclidean Distance 
 3 = normalized euclidean distance. 
Choose the value accordingly
2
 
We are doing the cases for k = 1, 3, 5, 7. Choose one of these values
5

Chosen distance_value is 2
Chosen k value is 5
0.9428571428571428
